In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score

pd.options.display.max_columns = 100
pd.options.display.max_rows = 150

In [2]:
df_train = pd.read_csv("data/competition_data/train.csv")
df_train = df_train.sample(frac=1).reset_index(drop=True)

df_test = pd.read_csv("data/competition_data/test.csv")

df_train

,id,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,nom_5,nom_6,nom_7,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month,target
0,242493,1,0,0,F,Y,Blue,Star,Dog,China,Theremin,4e28ceffa,b8caba53f,b0304151b,90dab3dd3,e77de7512,1,Novice,Cold,a,C,lF,4,1,0
1,23246,0,0,1,F,Y,Red,Trapezoid,Dog,Canada,Piano,1e6cb96e8,d213f48ed,c0a98f200,97dbfb7e6,92891527d,1,Master,Freezing,e,R,CZ,1,3,0
2,591,1,0,0,T,Y,Blue,Polygon,Lion,Russia,Theremin,2979f0d45,a20513779,6249769ec,f11f759db,ab5f3eee8,1,Contributor,Freezing,b,H,Os,5,1,0
3,13078,0,1,1,F,Y,Blue,Circle,Dog,China,Oboe,bd40c81f3,882ce2ad7,4a7761d5f,df8999dfb,f5bc8664b,1,Contributor,Boiling Hot,l,Q,yc,7,3,0
4,201881,0,0,0,T,Y,Red,Star,Cat,Costa Rica,Oboe,bd7740b93,0ef3dda96,e494c08d6,6d2a862e0,1afd5436e,1,Novice,Freezing,i,I,PQ,4,12,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299995,252400,1,0,0,F,N,Blue,Trapezoid,Cat,Russia,Bassoon,5b0f5acd5,944239e54,48e1b555b,baf4e8214,c4e25f270,1,Master,Freezing,a,Y,qA,3,1,0
299996,169387,0,0,1,F,Y,Blue,Star,Dog,Russia,Theremin,59e434d8a,e550bcc1b,e7ca9f823,6da888acf,f88695eff,3,Novice,Freezing,g,A,KR,2,10,0
299997,210998,0,0,1,F,Y,Blue,Square,Cat,Finland,Oboe,c2843dc45,4daee3baf,4ca8c6fdb,c9165c84a,376d38af6,1,Grandmaster,Lava Hot,i,S,uJ,2,11,0
299998,127486,0,0,1,T,N,Red,Trapezoid,Lion,Russia,Oboe,a5c276589,3bac0bd8e,055e2ceaa,1b812569c,2870d90b9,1,Expert,Cold,j,U,LE,2,9,1


In [3]:
y_train = df_train["target"]

y_train

0         0
1         0
2         0
3         0
4         0
         ..
299995    0
299996    0
299997    0
299998    1
299999    0
Name: target, Length: 300000, dtype: int64

# Logistic Regression on category codes

Expected to perform bad. Used as a benchmark for later one hot encoding

In [4]:
from sklearn.preprocessing import MinMaxScaler
    
def process_features_cats_to_codes(df):
    df_out = df.copy()
    
    # binary features to 0, 1
    df_out["bin_3"] = df_out["bin_3"].map({"T": 1, "F": 0})
    df_out["bin_4"] = df_out["bin_4"].map({"Y": 1, "N": 0})

    # ordinal features to int
    for field in ["ord_1", "ord_2", "ord_3", "ord_4", "ord_5"]:
        df[field] = pd.Categorical(df[field])
        df_out[field] = df[field].cat.codes
        
    # nominal features to int (same as for ordinal, but let's separate to keep us some options)
    for field in ['nom_0', 'nom_1', 'nom_2', 'nom_3', 'nom_4', 'nom_5', 'nom_6', 'nom_7', 'nom_8', 'nom_9']:
        df[field] = pd.Categorical(df[field])
        df_out[field] = df[field].cat.codes
        
    # scale the feature
    scaler = MinMaxScaler()
    df_out = pd.DataFrame(scaler.fit_transform(df_out), columns=df_out.columns)
    
    return df_out


df_train_codes = process_features_cats_to_codes(df_train)

del df_train_codes["target"]
del df_train_codes["id"]

df_train_codes

,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,nom_5,nom_6,nom_7,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month
0,1.0,0.0,0.0,0.0,1.0,0.0,0.6,0.4,0.2,1.000000,0.334842,0.710173,0.665299,0.565944,0.904841,0.0,1.00,0.2,0.000000,0.08,0.722513,0.500000,0.000000
1,0.0,0.0,1.0,0.0,1.0,1.0,0.8,0.4,0.0,0.666667,0.122172,0.804223,0.731747,0.593044,0.565943,0.0,0.75,0.4,0.285714,0.68,0.057592,0.000000,0.181818
2,1.0,0.0,0.0,1.0,1.0,0.0,0.2,0.8,1.0,1.000000,0.144796,0.629559,0.372436,0.943541,0.661853,0.0,0.00,0.4,0.071429,0.28,0.261780,0.666667,0.000000
3,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.4,0.2,0.333333,0.746606,0.518234,0.264971,0.874887,0.961269,0.0,0.00,0.0,0.785714,0.64,0.994764,1.000000,0.181818
4,0.0,0.0,0.0,1.0,1.0,1.0,0.6,0.2,0.4,0.333333,0.751131,0.051823,0.892535,0.429539,0.106678,0.0,1.00,0.4,0.571429,0.32,0.272251,0.500000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299995,1.0,0.0,0.0,0.0,0.0,0.0,0.8,0.2,1.0,0.000000,0.407240,0.564299,0.260870,0.728094,0.761686,0.0,0.75,0.4,0.000000,0.96,0.811518,0.333333,0.000000
299996,0.0,0.0,1.0,0.0,1.0,0.0,0.6,0.4,1.0,1.000000,0.393665,0.890595,0.907301,0.431346,0.972538,1.0,1.00,0.4,0.428571,0.00,0.178010,0.166667,0.818182
299997,0.0,0.0,1.0,0.0,1.0,0.0,0.4,0.2,0.6,0.333333,0.778281,0.305182,0.273175,0.783650,0.214942,0.0,0.50,0.8,0.571429,0.72,0.910995,0.166667,0.909091
299998,0.0,0.0,1.0,1.0,0.0,1.0,0.8,0.8,1.0,0.333333,0.683258,0.238004,0.028712,0.120145,0.158514,0.0,0.25,0.2,0.642857,0.80,0.198953,0.166667,0.727273


In [5]:
from sklearn.linear_model import LogisticRegression

In [6]:
# for training benchmark simply split training set in half

i = df_train_codes.shape[0]
df_train_codes1 = df_train_codes.copy()[:i // 2]
df_train_codes2 = df_train_codes.copy()[i // 2:]
y_train1=y_train[:i // 2]
y_train2=y_train[i // 2:]

In [7]:
model = LogisticRegression()

def fit_and_score(model, df_train, y_train, df_test, y_test):
    model = model.fit(df_train, y_train)
    
    y_score_train = [s[1] for s in model.predict_proba(df_train)]
    y_score_test = [s[1] for s in model.predict_proba(df_test)]

    print(f"""
        Score on training set: {roc_auc_score(y_true=y_train, y_score=y_score_train)}
        Score on test set: {roc_auc_score(y_true=y_test, y_score=y_score_test)}
    """)
    
    return model

model = fit_and_score(model, df_train_codes1, y_train1, df_train_codes2, y_train2)



        Score on training set: 0.7171347702073054
        Score on test set: 0.7164765605907126
    


In [8]:
"""Ok, good start, let's do one-hot encoding"""

"Ok, good start, let's do one-hot encoding"

# Logistic Regression on One Hot encoded features

In [9]:
from sklearn.preprocessing import OneHotEncoder

In [99]:
df_train_oh = df_train.copy()
df_test_oh = df_test.copy()

del df_train_oh["id"]
del df_train_oh["target"]
del df_test_oh["id"]

oh = OneHotEncoder()

oh.fit(pd.concat([df_train_oh, df_test_oh]))

df_train_oh = oh.transform(df_train_oh)
y_train_oh = y_train[:x]

df_train_oh.shape

(300000, 16552)

In [100]:
model = LogisticRegression(
    max_iter = 400,
    penalty='elasticnet',
    l1_ratio=1,
    C=.1,
    solver='saga',
    verbose=3
)

model = fit_and_score(
    model,
    df_train_oh[:x // 2],
    y_train_oh[:x // 2],
    df_train_oh[x // 2:],
    y_train_oh[x // 2:]
)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


convergence after 28 epochs took 19 seconds


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   19.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   19.7s finished



        Score on training set: 0.7958585177799675
        Score on test set: 0.7873276143608884
    


In [88]:
"""Looks nice, let's submit"""

"Looks nice, let's submit"

In [101]:
model = LogisticRegression(
    max_iter = 400,
    penalty='elasticnet',
    l1_ratio=1,
    C=.1,
    solver='saga',
    verbose=3
)

model = fit_and_score(
    model,
    df_train_oh,
    y_train_oh,
    df_train_oh,
    y_train_oh
)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


convergence after 26 epochs took 54 seconds


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   54.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   54.1s finished



        Score on training set: 0.8027193895415556
        Score on test set: 0.8027193895415556
    


In [103]:
submission = pd.read_csv("data/competition_data/sample_submission.csv")

df_test_oh = oh.transform(df_test_oh)

y_sub = model.predict_proba(df_test_oh)

In [105]:
submission["target"] = [x[1] for x in y_sub]

submission

,id,target
0,300000,0.294059
1,300001,0.685673
2,300002,0.147196
3,300003,0.392234
4,300004,0.835211
...,...,...
199995,499995,0.271626
199996,499996,0.172099
199997,499997,0.236951
199998,499998,0.473648


In [107]:
submission.to_csv("data/submissions/one_hot_logistic.csv", header=True, index=False)

In [109]:
"""
kaggle score: 0.79897, I am satisfied. 

Top on leaderboard is 0.80283. I guess I could get closer to that with some fine tuning, but meh...
"""

'\nkaggle score: 0.79897, I am satisfied. \n\nTop on leaderboard is 0.80283. I guess I could get closer to that with some fine tuning, but meh...\n'

# No Success: one hot the nominals, linear the ordinals

In [70]:
df_train_ord = df_train.copy()[['ord_0', 'ord_1', 'ord_2', 'ord_3', 'ord_4', 'ord_5']]
df_train_oh_nom = df_train.copy()[
    ['bin_0', 'bin_1', 'bin_2', 'bin_3', 'bin_4', 'nom_0', 'nom_1',
       'nom_2', 'nom_3', 'nom_4', 'nom_5', 'nom_6', 'nom_7', 'nom_8', 'nom_9', 'day', 'month']
]

oh = OneHotEncoder()

train_oh_nom = oh.fit_transform(df_train_oh_nom)
y_train_oh = y_train[:x]

train_oh_nom.shape

(300000, 16214)

In [71]:
"""TODO: encode the ordinaly to ascending ints and scale to [0, 1]"""

for field in ["ord_3", "ord_4", "ord_5"]:
    df_train_ord = df_train_ord.sort_values(by=field)
    df_train_ord[field] = df_train_ord[field].sort_values().cat.codes
    df_train_ord[field] = df_train_ord[field] / df_train_ord[field].max()
    
df_train_ord["ord_0"] = df_train_ord["ord_0"] / df_train_ord["ord_0"].max()

df_train_ord["ord_1"] = df_train_ord["ord_1"].map({
    'Grandmaster': 1, 
    'Novice': 0,
    'Master': 0.75,
    'Contributor': 0.25,
    'Expert': 0.5
})

df_train_ord["ord_2"] = df_train_ord["ord_2"].map({
    'Freezing': 0,
    'Warm': 0.4,
    'Lava Hot': 1,
    'Cold': 0.2,
    'Boiling Hot': 0.8,
    'Hot': 0.6
})

train_ord = df_train_ord.values
train_ord

array([[0.33333333, 1.        , 0.        , 0.5       , 0.8       ,
        0.        ],
       [0.33333333, 0.        , 0.4       , 0.35714286, 0.84      ,
        0.        ],
       [0.33333333, 0.        , 1.        , 0.28571429, 0.04      ,
        0.        ],
       ...,
       [0.33333333, 0.25      , 0.4       , 0.        , 0.        ,
        1.        ],
       [0.33333333, 0.        , 0.        , 0.21428571, 0.84      ,
        1.        ],
       [0.66666667, 0.        , 0.8       , 0.28571429, 0.72      ,
        1.        ]])

In [83]:
from scipy.sparse import hstack

train_sparse = hstack((train_oh_nom, train_ord))

train_sparse = train_sparse.tocsr()

train_sparse

<300000x16220 sparse matrix of type '<class 'numpy.float64'>'
	with 6617453 stored elements in Compressed Sparse Row format>

In [86]:
model = LogisticRegression(
    max_iter = 400,
    penalty='elasticnet',
    l1_ratio=.5,
    C=.1,
    solver='saga',
    verbose=3
)

model = fit_and_score(
    model,
    train_sparse[:x // 2],
    y_train_oh[:x // 2],
    train_sparse[x // 2:],
    y_train_oh[x // 2:]
)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


max_iter reached after 957 seconds


/home/wolle/miniconda3/envs/data_science/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 16.0min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 16.0min finished



        Score on training set: 0.7098635011884724
        Score on test set: 0.6868215532448938
    


In [87]:
"""Bad idea it seems"""

'Bad idea it seems'